In [1]:
import sys

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, KernelPCA
from sklearn.metrics import accuracy_score, matthews_corrcoef
from sklearn.model_selection import cross_val_score, cross_validate, KFold

from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import EditedNearestNeighbours

from xgboost import XGBClassifier, XGBRFClassifier

sys.path.append("../../../")

from helpers.split import make_train_test_split, tag_label_feature_split
from helpers.assess import make_classification_report, make_confusion_matrix

DATASET_FOLDER = "../../../datasets/"

/Users/rerobbins/opt/anaconda3/envs/tf/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


ModuleNotFoundError: No module named 'helpers'

In [ ]:
# read a data set
df = pd.read_pickle(DATASET_FOLDER + .pickle")

In [ ]:
# get labels, a label encoder and features
_, (y, le), X = tag_label_feature_split(df, label_format="encoded")

In [ ]:
pd.Series(y).value_counts()

In [ ]:
# review the shapes
print(f"{X.shape= } {y.shape= }")

In [ ]:
# split the data for training and testing with shuffling and stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=10, shuffle=True, stratify=y, 
)

In [ ]:
# review the shapes
print(f"{X_train.shape = }\t{y_train.shape = }")
print(f"{X_test.shape =  }\t{y_test.shape =  }")

In [ ]:
plt.figure(figsize=(8,4))
plt.hist([y, y_train, y_test],
         label=['y', 'y_train', 'y_test'])
plt.xlabel("Genre ID")
plt.legend()
plt.show()

In [ ]:
scaler=StandardScaler()
#pca=KernelPCA(random_state=1962, n_components=200, n_jobs=-1)
#pca=PCA(random_state=1962, n_components=.85)
#pipe = Pipeline([('scaler', scaler),
#                 ('pca', pca)
#                ])

#No pipe -- just scale the features
pipe=scaler

In [ ]:
# run the scaling/PCA extraction pipe
X_train_pca = pipe.fit_transform(X_train)
X_test_pca = pipe.transform(X_test)

In [ ]:
# review the shapes
print(f"{X_train_pca.shape= } {X_test_pca.shape= }")

In [ ]:
# Use SMOTE or SMOTE variant to resample the training data

#smt = SMOTE (random_state=1962, n_jobs=-1)

#enn = EditedNearestNeighbours(kind_sel='mode', n_neighbors=3, n_jobs=-1)
#smt = SMOTEENN (random_state=1962, enn=enn)

#smt = SMOTETomek(random_state=1962, n_jobs=-1)

#X_train_resampled, y_train_resampled = smt.fit_resample(X_train_pca, y_train)

# Pass through, no resampling, this is our baseline
X_train_resampled, y_train_resampled = X_train_pca, y_train

In [ ]:
unique, counts = np.unique(y_train_resampled, return_counts=True)
print(np.asarray((unique, counts)).T)

In [ ]:
plt.figure(figsize=(8,4))
plt.hist([y_train, y_train_resampled],
         label=['y_train', 'y_train_resampled'])
plt.xlabel("Genre ID")
plt.legend()
plt.show()

**Preliminary Baseline**

In [ ]:
model = XGBClassifier(
    use_label_encoder=False,
    tree_method='gpu_hist', 
    eta=.05,
    max_depth=6,
    reg_alpha=10,
    sampling_method='gradient_based',
    objective="multi:softprob", 
    eval_metric="mlogloss",
    seed=1962,
)

In [ ]:
# Let's just fit the model on our training data
model.fit(X_train_resampled, y_train_resampled)

In [ ]:
scores = 
print("Mean train cross-validation mcc score: %.4f" % scores.mean())

In [ ]:
y_pred=model.predict(X_test_pca)

In [ ]:
import json
import warnings

scoring=['matthews_corrcoef', 'f1_macro', 'accuracy', 'precision_macro', 'recall_macro']

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    scores = cross_validate (model, X_train_resampled, y_train_resampled, scoring=scoring, cv=2, error_score=0)

scores

In [ ]:
{key.strip("test_"): value.mean() for key, value in scores.items() if "test" in key}

In [ ]:
import json
json.dumps({key.strip("test_"): value.mean() for key, value in scores.items() if "test" in key})

In [ ]:
print(f"Training accuracy")
make_classification_report(
    y_train_resampled,
    model=model,
    x=X_train_resampled,
    digits=4,
    label_encoder=le,
    print_report=True,
    save_result=True,
    result_filename="rich_xgboost_results.json",
    model_name="XGBClassifier_baseline",
    dataset_name="dataset_00_all",
    phase="train",
    repeat=True,
    )
print(f"\nTesting accuracy")
make_classification_report(
    y_test,
    y_pred=y_pred,
    digits=4,
    label_encoder=le,
    print_report=True,
    save_result=True,
    result_filename="rich_xgboost_results.json",
    model_name="XGBClassifier_baseline",
    dataset_name="dataset_00_all",
    phase="test",
    repeat=True,
    )
pass

In [ ]:
dataset_name="Preliminary Baseline"
make_confusion_matrix(
    y_test,
    y_pred=y_pred,
    label_encoder=le,
    title=f"{dataset_name} test (row normalized)",
)
pass

**Optimized Model**

In [ ]:
model = XGBClassifier(
    
    learning_rate= 0.11484036953359722,
    gamma = 0.327162074525032, 
    max_depth = 8, 
    n_estimators = 427, 
    reg_alpha= 6.88045418099, 
    use_label_encoder=False,
    tree_method='gpu_hist', 
    sampling_method='gradient_based',
    objective="multi:softprob", 
    eval_metric="mlogloss",
    seed=1962,
)

In [ ]:
model.fit(X_train_resampled, y_train_resampled)

In [ ]:
scores = cross_val_score(model, X_train_resampled, y_train_resampled, 
                         scoring="matthews_corrcoef", cv=5,  n_jobs=-1, )
print("Mean train cross-validation mcc score: %.4f" % scores.mean())

In [ ]:
y_pred=model.predict(X_test_pca)

In [ ]:
print(f"Training accuracy")
make_classification_report(
    y_train_resampled,
    model=model,
    x=X_train_resampled,
    digits=4,
    label_encoder=le,
    print_report=True,
    save_result=True,
    result_filename="rich_xgboost_results.json",
    model_name="XGBClassifier_baseline_tuned",
    dataset_name="dataset_00_all",
    phase="train",
    repeat=True,
    )
print(f"\nTesting accuracy")
make_classification_report(
    y_test,
    y_pred=y_pred,
    digits=4,
    label_encoder=le,
    print_report=True,
    save_result=True,
    result_filename="rich_xgboost_results.json",
    model_name="XGBClassifier_baseline_tuned",
    dataset_name="dataset_00_all",
    phase="test",
    repeat=True,
    )
pass

In [ ]:
dataset_name="Optimized Baseline"
make_confusion_matrix(
    y_test,
    y_pred=y_pred,
    label_encoder=le,
    title=f"{dataset_name} test (row normalized)",
)
pass